In [2]:
import datetime as dt
from utility import (
    read_sql,
    fd_basicinfo,
    fd_typeclass,
    nearest_report_date
)
import pandas as pd

In [3]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [4]:
# 获取所有目标债券型基金（清算结束日期晚于当前日）

security_ids = read_sql(f"""
select SECURITYID from TQ_FD_TYPECLASS 
WHERE
    ISVALID = 1 AND
    L1CODE = 3 AND
    (ENDDATE>='{report_dates_begin}' or ENDDATE = '19000101') 
ORDER BY SECURITYID;
""")["SECURITYID"].unique().tolist()
sec_id_strs = ",".join(["'" + s + "'" for s in security_ids])

not_liq_sec_ids = fd_basicinfo(security_ids, current_date)["SECURITYID"].unique().tolist()
sec_id_strs = ",".join(["'" + s + "'" for s in not_liq_sec_ids])

In [5]:
# 获取相关组合情况

query = f"""
SELECT SECURITYID, REPORTDATE, BDRTO, CONVBDRTO, EQUITYINVERTO from TQ_FD_ASSETPORTFOLIO
WHERE
    REPORTDATE >= '{report_dates_begin}' AND
    ISVALID = 1 AND
    SECURITYID in ({sec_id_strs}) ORDER BY SECURITYID, REPORTDATE
"""
portfolios = read_sql(query).fillna(0.0)
portfolios["CONVBDRTO_TO_BDRTO"] = portfolios["CONVBDRTO"] / portfolios["BDRTO"] * 100

In [6]:
last_portfolio = portfolios.groupby("SECURITYID").last()[["BDRTO"]]
last_4_cov_portfolio = portfolios.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
last_4_stk_portfolio = portfolios.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

In [15]:
agg_port = pd.concat([last_portfolio, last_4_cov_portfolio, last_4_stk_portfolio], axis=1).reset_index()
agg_port = pd.merge(agg_port, fd_typeclass(agg_port.SECURITYID.tolist(), current_date))
basic_info = fd_basicinfo(agg_port.SECURITYID.tolist())
agg_port = pd.merge(agg_port, basic_info)

In [17]:
agg_port.L3NAME.unique()

array(['普通债券型基金(一级A类)', '普通债券型基金(一级B/C类)', '普通债券型基金(二级A类)',
       '普通债券型基金(二级B/C类)', '中短期标准债券型基金', '普通债券型基金(可投转债A类)', '保本型基金',
       '普通债券型基金(可投转债B类)', '指数债券型基金(A类)', '封闭式普通债券型基金(一级)(A类)',
       '长期标准债券型基金(A类)', '封闭式普通债券型基金(二级)(A类)', '可转换债券型基金(A类)',
       '可转换债券型基金(B/C类)', '债券型分级子基金(优先份额)', '债券型分级子基金(进取份额)',
       '指数债券型基金(B/C类)', '封闭式债券型分级子基金(优先份额)', '封闭式长期标准债券型基金(A类)',
       '长期标准债券型基金(B/C类)', '短期理财债券型基金(A类)', '短期理财债券型基金(B/C类)',
       '封闭式普通债券型基金(可投转债)(A类)', '灵活策略基金（A类）', '封闭式长期标准债券型基金(B/C类)',
       '灵活配置型基金(股票上限95%)（A类）', '灵活配置型基金(股票上限95%)（B/C类）'], dtype=object)

In [24]:
# 分类
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 5) & (agg_port["CONVBDRTO_TO_BDRTO"] < 80) & (agg_port["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["EQUITYINVERTO"] >= 0.1) & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["L2NAME"] == "指数债券型基金") & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
agg_port.loc[pd.isnull(agg_port["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"

In [26]:
agg_port.groupby("债券型（子类）").count()

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,663,663,648,657,663,663,663,663,663,663,663,466,663,663,663
可投股票型债券基金,403,403,403,403,403,403,403,403,403,403,403,349,403,403,403
可投转债型债券基金,89,89,89,89,89,89,89,89,89,89,89,82,89,89,89
可转债基金,52,52,52,52,52,52,52,52,52,52,52,41,52,52,52
短期纯债型基金,65,65,65,65,65,65,65,65,65,65,65,64,65,65,65
被动指数型债券基金,17,17,17,17,17,17,17,17,17,17,17,11,17,17,17
